# ISAF (إسعاف)  
## Integrated Security Assessments for your dev Flow
```
    EeeiiiiiEEiiiii.....                                             
       \|/                                                           
        n______     .....iiiiiEEiiiieeEE                             
       :~;     :                  \|/                                
-----;``~'  +  ;------------ ______n --------------------------------
     `-@-----@-=            :     :~:                                
=========================== ;  +  '~``; =============================
                            =-@-----@-'                              
jgs------------------------------------------------------------------
                                                                     
                   DEVSECOPS IN A PYTHON NUTSHELL        
```

## Environment Build

### What App?
A tiny vulnerable application written in PHP.
[![Vulnerable App](screen_webapp.png)](https://github.com/h-a-t/RedPill/blob/master/src/php/index.php)    
These 2 lines are vulnerable to SQL injections and to XSS attacks. 
```php
    $user_id = $_GET['id'];
    $sql = mysql_query("SELECT username, nom, prenom, email FROM users WHERE user_id = $user_id") or die(mysql_error());
```

### Let's deploy it!

In [ ]:
!git clone https://github.com/h-a-t/RedPill

In [ ]:
!pip install docker

In [4]:
import docker
import io
import tarfile
import os

cli = docker.from_env()
cli.containers.list()

In [ ]:
## Build app image and pull dependencies
os.chdir('/home/jovyan/RedPill/')
cli.images.build(path='./src/php', tag='hat/app')
#zap_img = cli.images.pull('owasp/zap2docker-weekly:latest')
db_img = cli.images.pull('mariadb:latest')
alpine = cli.images.pull('alpine:latest')
cli.images.list()

In [ ]:
## Create a dedicated network
cli.networks.create("app_net", driver="overlay")
for x in cli.networks.list():
    print("%s  %s" % (x.id,x.name))

In [ ]:
## Create Volumes
cli.volumes.create(name='db_data', driver='local')
cli.volumes.create(name='db_init', driver='local')
cli.volumes.create(name='app_data', driver='local')

In [ ]:
## Database provisionning
os.chdir('/home/jovyan/RedPill/src/sql')
tarstream = io.BytesIO()
tar = tarfile.TarFile(fileobj=tarstream, mode='w')
tar.add('staging.sql')
tar.close()

## https://gist.github.com/zbyte64/6800eae10ce082bb78f0b7a2cca5cbc2
tmp=cli.containers.create(
    image='alpine', 
    volumes={'db_init':{'bind': '/data/', 'mode' : 'rw'}})

tarstream.seek(0)
tmp.put_archive(
    path='/data/',
    data=tarstream
)

## Database run
db_cont = cli.services.create(
    image='mariadb:latest',
    mounts=[
        "db_init:/docker-entrypoint-initdb.d/:rw",
        "db_data:/var/lib/mysql/:rw"
    ],
    networks= ['app_net'],
    name='app_db',
    env=['MYSQL_RANDOM_ROOT_PASSWORD=yes','MYSQL_USER=user',
                  'MYSQL_PASSWORD=password','MYSQL_DATABASE=sqli']
        )
db_cont?

In [ ]:
## Webserver provisionning

os.chdir('/home/jovyan/RedPill/src/php')
tarstream = io.BytesIO()
tar = tarfile.TarFile(fileobj=tarstream, mode='w')
tar.add('.')
tar.close()

tmp=cli.containers.create(
    image='alpine', 
    volumes={'app_data':{'bind': '/data/', 'mode' : 'rw'}})

tarstream.seek(0)
tmp.put_archive(
    path='/data/',
    data=tarstream
)
## webserver run
app_cont = cli.services.create(
    image='hat/app',
    mounts=[
        "app_data:/var/www/html:rw",
    ],
    networks= ['app_net'],
    name='app_web',
    endpoint_spec={
                'Mode': 'vip',
                "Ports": [
                    {
                        "Protocol": "tcp",
                        "TargetPort": 80,
                        "PublishedPort": 80
                    }]
                    },
    env=['DB_ENV_MYSQL_USER=user','DB_ENV_MYSQL_PASSWORD=password','BUILD_STAGE=Python'],
        )
app_cont?

# Security Assessment in a Synchronous Execution Flow

## Static Code Analysis


### Push code to SonarQube for code analysis

In [ ]:
## Download Sonarqube scanner
!wget https://sonarsource.bintray.com/Distribution/sonar-scanner-cli/sonar-scanner-cli-3.0.3.778-linux.zip
!unzip sonar-scanner-cli-3.0.3.778-linux.zip

In [ ]:
!./sonar-scanner-3.0.3.778-linux/bin/sonar-scanner -Dsonar.host.url=http://sonarqube:9000 -Dsonar.projectKey=Redpill:latest -Dsonar.sources=RedPill/src/php -Dsonar.language=php

### Push to Clair for a container layer scan

TODO : reg de Jess + Pop instance Clair

## Dynamic Runtime Analysis

### Using the Web-GUI of your favorite Pentesting tool from OWASP: ZAP
- Let L33t do the testing by running a GUI instance of ZAP, just browse localhost:8666/?anonym=true&app=ZAP to start :) 
- Warning, image size: 1,52 Go  
- Cf. https://github.com/zaproxy/zaproxy/wiki/WebSwing  

In [ ]:
zap_img = cli.images.pull('owasp/zap2docker-stable:latest')
scan_cont = cli.containers.run(
    image=zap_img,
    name='app_scan',
    detach=True,
    command="sh -c 'zap-webswing.sh'",
    ports={'8080/tcp':8666, '8090/tcp':8777},
    links=[('app_web','app')]
        )
scan_cont?

### Using the flexibility of your favorite cutting-edge technology: OpenFaas

Dockerfile:  

```
FROM alexellis2/faas-alpinefunction:latest
RUN apk update && apk add nmap
ENV fprocess="xargs nmap"
CMD ["fwatchdog"]
```

nmap_stack:   

```yaml
provider:
  name: faas
  gateway: http://gateway:8080

functions:
  nmap:
    lang: Dockerfile
    handler: ./Dockerfile
    image: hat/nmap
```

In [ ]:
import tarfile
import time
from io import BytesIO
os.chdir('/home/jovyan/work')
Dockerfile ='''
FROM alexellis2/faas-alpinefunction:latest
RUN apk update && apk add nmap
ENV fprocess="xargs nmap"
CMD ["fwatchdog"]
'''
with open("Dockerfile", "w") as stack:
    stack.write("%s" % Dockerfile)

#write the Dockerfile to a tarred archive
pw_tarstream = BytesIO()
pw_tar = tarfile.TarFile(fileobj=pw_tarstream, mode='w')
file_data = Dockerfile.encode('utf8')
tarinfo = tarfile.TarInfo(name='Dockerfile')
tarinfo.size = len(file_data)
tarinfo.mtime = time.time()
#tarinfo.mode = 0600
pw_tar.addfile(tarinfo, BytesIO(file_data))
pw_tar.close()

pw_tarstream.seek(0)
nmap = cli.images.build(
    fileobj=pw_tarstream,
    custom_context=True,
    tag='hat/nmap'
)
nmap?

In [ ]:
## nmap Stack
os.chdir('/home/jovyan/work')
func_stack='''
provider:
  name: faas
  gateway: http://gateway:8080

functions:
  nmap:
    lang: Dockerfile
    handler: ./Dockerfile
    image: hat/nmap
'''
with open("nmap_func.yml", "w") as stack:
    stack.write("%s" % func_stack)

In [ ]:
!faas-cli build -f nmap_func.yml

In [ ]:
!faas-cli deploy -f nmap_func.yml

In [ ]:
## Testing
!curl -v http://gateway:8080/system/functions

In [ ]:
## Executing nmap \o/
!curl -v --data "-T4 172.17.0.3" http://gateway:8080/function/nmap

Async IO with docker
-> faas in python
https://pypi.python.org/pypi/aiodocker/0.8.2
http://aiodocker.readthedocs.io/en/latest/
https://curio.readthedocs.io/en/latest/tutorial.html

In [ ]:
import asyncio
from aiodocker.docker import Docker
from aiodocker.exceptions import DockerError


async def demo(docker):
    try:
        await docker.images.get('alpine:latest')
    except DockerError as e:
        if e.status == 404:
            await docker.pull('alpine:latest')
        else:
            print('Error retrieving alpine:latest image.')
            return

    config = {
        # "Cmd": ["/bin/ash", "-c", "sleep 1; echo a; sleep 1; echo a; sleep 1; echo a; sleep 1; echo x"],
        "Cmd": ["/bin/ash"],
        "Image": "alpine:latest",
        "AttachStdin": True,
        "AttachStdout": True,
        "AttachStderr": True,
        "Tty": False,
        "OpenStdin": True,
        "StdinOnce": True,
    }
    container = await docker.containers.create_or_replace(
        config=config, name='aiodocker-example')
    print("created and started container {}".format(container._id[:12]))

    try:
        ws = await container.websocket(stdin=True, stdout=True, stderr=True, stream=True)
        await container.start()

        async def _send():
            await asyncio.sleep(0.5)
            await ws.send_bytes(b'echo "hello world"\n')
            print("sent a shell command")

        asyncio.ensure_future(_send())
        resp = await ws.receive()
        print("received: {}".format(resp))
        await ws.close()

        output = await container.log(stdout=True)
        print("log output: {}".format(output))
    finally:
        print("removing container")
        await container.delete(force=True)


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    docker = Docker()
    try:
        loop.run_until_complete(demo(docker))
    finally:
        loop.run_until_complete(docker.close())
        loop.close()

In [ ]:
import asyncio
from aiodocker.docker import Docker


async def demo(docker):
    print('--------------------------------')
    print('- Check Docker Version Information')
    data_version = await docker.version()
    for key, value in data_version.items():
        print(key, ':', value)

    print('--------------------------------')
    print('- Check Docker Image List')
    images = await docker.images.list()
    for image in images:
        for key, value in image.items():
            if key == 'RepoTags':
                print(key, ':', value)

    print('--------------------------------')
    print('- Check Docker Container List')
    containers = await docker.containers.list()
    for container in containers:
        container_show = await container.show()
        for key, value in container_show.items():
            if key == 'Id':
                print('Id', ':', value[:12])
    print('--------------------------------')


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    docker = Docker()
    try:
        loop.run_until_complete(demo(docker))
    finally:
        loop.run_until_complete(docker.close())
        loop.close()

In [ ]:
import asyncio
from aiodocker.docker import Docker
from aiodocker.exceptions import DockerError


async def demo(docker):
    try:
        await docker.images.get('alpine:latest')
    except DockerError as e:
        if e.status == 404:
            await docker.pull('alpine:latest')
        else:
            print('Error retrieving alpine:latest image.')
            return

    subscriber = docker.events.subscribe()

    config = {
        "Cmd": ["tail", "-f", "/var/log/dmesg"],
        "Image":"alpine:latest",
         "AttachStdin": False,
         "AttachStdout": True,
         "AttachStderr": True,
         "Tty": False,
         "OpenStdin": False,
         "StdinOnce": False,
    }
    container = await docker.containers.create_or_replace(
        config=config, name='testing')
    await container.start()
    print("=> created and started container {}".format(container._id[:12]))

    while True:
        event = await subscriber.get()
        if event is None:
            break

        for key, value in event.items():
            print(key,':', value)

        # Demonstrate simple event-driven container mgmt.
        if event['Actor']['ID'] == container._id:
            if event['Action'] == 'start':
                await container.stop()
                print("=> killed {}".format(container._id[:12]))
            elif event['Action'] == 'stop':
                await container.delete(force=True)
                print("=> deleted {}".format(container._id[:12]))
            elif event['Action'] == 'destroy':
                print('=> done with this container!')
                break

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    docker = Docker()
    try:
        # do our stuffs.
        loop.run_until_complete(demo(docker))
    finally:
        loop.run_until_complete(docker.close())
        loop.close()

In [ ]:
import asyncio
import aiodocker

async def list_things():
    docker = aiodocker.Docker()
    print('== Images ==')
    for image in (await docker.images.list()):
        tags = image['RepoTags'][0] if image['RepoTags'] else ''
        print(image['Id'], tags)
    print('== Containers ==')
    for container in (await docker.containers.list()):
        print(f" {container._id}")
    await docker.close()

async def run_container():
    docker = aiodocker.Docker()
    print('== Running a hello-world container ==')
    container = await docker.containers.create_or_replace(
        config={
            'Cmd': ['/bin/ash', '-c', 'echo "hello world"'],
            'Image': 'alpine:latest',
        },
        name='testing',
    )
    await container.start()
    logs = await container.log(stdout=True)
    print(''.join(logs))
    await container.delete(force=True)
    await docker.close()

if __name__ == '__main__':
    loop2 = asyncio.get_event_loop()
    loop2.run_until_complete(list_things())
    loop2.run_until_complete(run_container())
    loop2.close()